In [1]:
!pip install mlflow
!pip install pyngrok

In [2]:
from pyngrok import ngrok

# Kill all active tunnels
ngrok.kill()


In [4]:
import mlflow
import mlflow.sklearn

mlflow.set_experiment("Exp 4- Handling Imbalanced data")

from google.colab import userdata
ngRoktoken=userdata.get('ngroktoen')

get_ipython().system_raw('mlflow ui --port 2000 &')
mlflow.set_tracking_uri('http://localhost:2000')
from pyngrok import ngrok
ngrok.set_auth_token(ngRoktoken)

public_url=ngrok.connect(2000).public_url
print('mlflow ui url: ', public_url)

mlflow ui url:  https://8931-34-139-230-230.ngrok-free.app


In [5]:
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os


In [6]:
df=pd.read_csv('reddit_preprocessing.csv').dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [8]:
# Step 1: Function to run the experiment
def run_imbalanced_experiment(imbalance_method):
    ngram_range = (1, 3)  # Trigram setting
    max_features = 1000   # set max_features to 1000 for TF-IDF

    # Step 2: Vectorization using TF-IDF
    vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
    X = vectorizer.fit_transform(df['clean_comment'])
    y = df['category']

    # Step 3: Handle class imbalance based on the selected method
    if imbalance_method == 'class_weights':
        class_weight = 'balanced'
    else:
        class_weight = None  # do not apply class_weight if using resampling

        # resampling techniques
        if imbalance_method == 'oversampling':
            smote = SMOTE(random_state=42)
            X, y = smote.fit_resample(X, y)
        elif imbalance_method == 'adasyn':
            adasyn = ADASYN(random_state=42)
            X, y = adasyn.fit_resample(X, y)
        elif imbalance_method == 'undersampling':
            rus = RandomUnderSampler(random_state=42)
            X, y = rus.fit_resample(X, y)
        elif imbalance_method == 'smote_enn':
            smote_enn = SMOTEENN(random_state=42)
            X, y = smote_enn.fit_resample(X, y)

    # Step 4: Train-test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    # Step 5: Define and train Random Forest model
    with mlflow.start_run() as run:
        # Set MLflow tags
        mlflow.set_tag("mlflow.runName", f"Imbalance_{imbalance_method}_RandomForest_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "imbalance_handling")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # add description
        mlflow.set_tag("description", f"Random Forest with TF-IDF Trigrams, imbalance method={imbalance_method}")

        # Log vectorizer and model parameters
        mlflow.log_param("vectorizer_type", "TF-IDF")
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", max_features)

        # log random forest
        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)
        mlflow.log_param("imbalance_method", imbalance_method)

        # initialize and train the model
        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            random_state=42,
            class_weight=class_weight
        )
        model.fit(X_train, y_train)

        # Step 6: Make predictions and log metrics
        y_pred = model.predict(X_test)

        # log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # log confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: TF-IDF Trigrams, Imbalance={imbalance_method}")
        plt.savefig("confusion_matrix.png")
        mlflow.log_artifact("confusion_matrix.png")
        plt.close()

        # Log the trained model with input example and signature
        from mlflow.models.signature import infer_signature

        input_example = X_test[:1]
        pred_example = model.predict(input_example)
        signature = infer_signature(input_example, pred_example)

        mlflow.sklearn.log_model(
            sk_model=model,
            artifact_path=f"random_forest_model_tfidf_trigrams_imbalance_{imbalance_method}",
            input_example=input_example,
            signature=signature
        )

# Step 7: Run experiments for different imbalance methods
imbalance_methods = ['class_weights', 'oversampling', 'adasyn', 'undersampling', 'smote_enn']

for method in imbalance_methods:
    run_imbalanced_experiment(method)


2025/06/28 07:18:25 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Imbalance_class_weights_RandomForest_TFIDF_Trigrams at: http://localhost:2000/#/experiments/436558268196892237/runs/1384209aed144e81b6f170373cbdc268
🧪 View experiment at: http://localhost:2000/#/experiments/436558268196892237


2025/06/28 07:18:52 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Imbalance_oversampling_RandomForest_TFIDF_Trigrams at: http://localhost:2000/#/experiments/436558268196892237/runs/226e4c8f633341d2b90a62ce533d6914
🧪 View experiment at: http://localhost:2000/#/experiments/436558268196892237


2025/06/28 07:19:38 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Imbalance_adasyn_RandomForest_TFIDF_Trigrams at: http://localhost:2000/#/experiments/436558268196892237/runs/43ba88b4c3b548b88a326ba67ae3761a
🧪 View experiment at: http://localhost:2000/#/experiments/436558268196892237


2025/06/28 07:19:55 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Imbalance_undersampling_RandomForest_TFIDF_Trigrams at: http://localhost:2000/#/experiments/436558268196892237/runs/85d07fae9fb1435f8ca5b48aaccf9504
🧪 View experiment at: http://localhost:2000/#/experiments/436558268196892237


2025/06/28 07:21:20 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Imbalance_smote_enn_RandomForest_TFIDF_Trigrams at: http://localhost:2000/#/experiments/436558268196892237/runs/763b9c235b784f5792621a76ba6eec9d
🧪 View experiment at: http://localhost:2000/#/experiments/436558268196892237
